In [ ]:
from config import load_conf

config = load_conf("./config.yaml")
nicknames = ' or '.join(config.my.names)
print(nicknames)

In [ ]:
from slack_util import SlackUtil

slack = SlackUtil(token=config.slack.token)
chat_history = []
channel_to_summary = config.slack.channels["your-channel"]

conversation_history = slack.client.conversations_history(
    channel=channel_to_summary,
    limit=20
)
for message in reversed(conversation_history["messages"]):
    chat = slack.to_chat_entry(message)
    print(chat)
    chat_history.append(chat)
# chat_history = slack.get_mock_chat_history()
# for chat in chat_history:
#     print(chat)


In [ ]:
# This is for ollama
# from langchain_community.llms import Ollama
# llm = Ollama(model="llama3")
# llm = Ollama(model="phi3")
# llm = Ollama(model="wizardlm2")

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(api_key=config.open_ai.api_key,
                 model="gpt-3.5-turbo")

# llm = ChatOpenAI(api_key=config.open_ai.api_key,
#                  model="gpt-4o")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system",
     """
     I'd like you to analyze the following conversation and identify all my pending action items 
     that someone specifically asked me to do and I'm not resolved or fulfilled and still require my attention.
     Others may address me by {nicknames} or <Me> in the conversation.
     Please format these action items as a bulleted list with clear and concise descriptions of what needs to be done.
     """),
    ("user",
     """
     {conversation}
     """),
])

In [ ]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

chain = prompt | llm | output_parser
output = chain.invoke({"conversation": chat_history, "nicknames": nicknames})
print(output)

In [ ]:
from typing_extensions import TypedDict

class State(TypedDict):
    conversation: list[str]
    action_items: str
    pending_items: str

In [ ]:
def get_conversation(state: State):
    state = {**state, "conversation": chat_history}
    return state

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

def get_action_runnable(state: State):
    get_action_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """
                I need you to analyze a conversation to extract action items that speaker specifically want me to take action.
                List them in this format: <Who> want me to <do something>.
                Others may address me by {nicknames} or <Me> in the conversation.
                Output the extracted action items and no preamble or explanation.
                """,
            ),
            ("user", "The conversations for analysis:\n{conversation}"),
        ]
    )
    output_parser = StrOutputParser()

    chain = get_action_prompt | llm | output_parser
    output = chain.invoke({"conversation": state["conversation"],
                           "nicknames": nicknames})
    state = {**state, "action_items": output}
    return state

def get_pending_action_runnable(state: State):
    get_pending_account_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """
                I have conversation and a list of action items that someone want me to do something.
                I need you to analyze the conversation and the identified action items 
                to find out which action items are still pending, not fulfill and require my attention.
                Others may address me by {nicknames} or <Me> in the conversation.
                No preamble or explanation""",
            ),
            (
                "user", 
                "Conversation: {conversation}\n"
                "Action Items: {action_items}\n"
             ),
        ]
    )
    output_parser = StrOutputParser()

    chain = get_pending_account_prompt | llm | output_parser
    output = chain.invoke({
        "conversation": state["conversation"],
        "action_items": state["action_items"],
        "nicknames": nicknames
        })
    state = {**state, "pending_items": output}
    return state

In [ ]:
from langgraph.graph import END, StateGraph

builder = StateGraph(State)

builder.add_node("get_chat_history", get_conversation)
builder.set_entry_point("get_chat_history")

builder.add_node("get_action", get_action_runnable)
builder.add_edge("get_chat_history", "get_action")

builder.add_node("get_pending_action", get_pending_action_runnable)
builder.add_edge("get_action", "get_pending_action")

builder.add_edge("get_pending_action", END)

graph = builder.compile()


In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph(xray=True).draw_mermaid_png()))
except Exception as e:
    # This requires some extra dependencies and is optional
    print(e)
    pass

In [ ]:
import uuid
from langchain_core.runnables import RunnableConfig
thread_id = str(uuid.uuid4())

config = RunnableConfig(
    configurable={
        "thread_id": thread_id})

event = graph.invoke({})
if event["action_items"] != None:
    print("------Action Items------")
    print(event["action_items"])
if event["pending_items"] != None:
    print("------Pending Items------")
    print(event["pending_items"])
